<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Twitter- Generate leads from post likes
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/LinkedIn_Get_comments_from_post.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

**Tags:** #twitter #post #comments #naas_drivers #automation

**Author:** [Alok Chilka](https://www.linkedin.com/in/calok64/)

## Input

### Import libraries

In [ ]:
!pip install --user tweepy

In [2]:
from naas_drivers import linkedin, hubspot
import pandas as pd
import numpy as np
import naas
from datetime import datetime, timedelta
import requests
import json
import tweepy

### Setup your Twitter

### How to get token ?

bearer_token – the token used for authentication
https://developer.twitter.com/en/docs/authentication/oauth-2-0/bearer-tokens


In [105]:
#Twitter cookies
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAFGZZgEAXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

client = tweepy.Client(bearer_token=bearer_token)
user_name = "<YOUR_PREFERRED_USERNAME>"
users = client.get_users(usernames=[user_name])
user_id = users.data[0]["id"]

#set application type. This is used to distinguish data for application in master data model
APP_TYPE = "Twitter"

BASE_URL = "https://twitter.com/"

### Setup your email to receive notifications

In [7]:
SEND_EMAIL_TO = "<YOUR_EMAIL_ID>"

## Model

### Get posts from Twitter feed

In [4]:
#Get user tweets
tweets = client.get_users_tweets(id=user_id)

### Build tweet id list

In [5]:
tweetIdList = []
for tweet in tweets.data:
    tweetIdList.append(str(tweet["id"]))

In [ ]:
tweetIdList

### Get likes from Twitter posts

In [124]:
def get_post_stats(tweetIdList):
    twitterDF = pd.DataFrame()
    for tweetId in tweetIdList:

        #print("*********************************")
        #print("Tweet ID : ", tweetId)
        parent_tweet = client.get_tweet(id=tweetId, tweet_fields=["author_id","created_at","entities","in_reply_to_user_id",
                                                                        "referenced_tweets,source,public_metrics"])
        parent_user_name = user_name
        parent_Tweet_Title = ""
        #parent_tweet_id = reply_tweet.data["data"]["referenced_tweets"][0]["id"]
        parent_tweet_text = parent_tweet.data["text"]
        parent_created_at = parent_tweet.data["created_at"].strftime("%d-%m-%Y %H:%M:%S")
        parent_tweet_URL = BASE_URL+user_name+"/status/"+tweetId
        parent_public_metrics = parent_tweet.data["data"]["public_metrics"]
        parent_retweet_count = parent_public_metrics["retweet_count"]
        parent_reply_count = parent_public_metrics["reply_count"]
        parent_like_count = parent_public_metrics["like_count"]
        parent_quote_count = parent_public_metrics["quote_count"]
        parent_view_count = 0 #currently twitter has no ways to find out who actually viewed your post hence kept value = 0 to map the columns
        parent_comments_mentions = []

        retweeted_by_users = client.get_retweeters(id=tweetId)
        
        retweeted_by_username = []
        if retweeted_by_users.data != None:
            for retweet_user in retweeted_by_users.data:
                retweeted_by_username.append(retweet_user["username"])
        
        like_by_users = client.get_liking_users(id=tweetId)        
        liked_by_username = []
        if like_by_users.data != None:
            for like_user in like_by_users.data:
                liked_by_username.append(like_user["username"])
        
        mentions = client.get_users_mentions(id=user_id)
        
        comments = []
        if mentions.data != None:
            for mention in mentions.data:
                child_tweet = client.get_tweet(id=mention["id"], tweet_fields=["author_id", "in_reply_to_user_id","referenced_tweets,source,public_metrics,text"])
                in_reply_to_tweet_id = child_tweet.data["data"]["referenced_tweets"][0]["id"]

                if str(in_reply_to_tweet_id) == tweetId:
                    username = "@"+user_name
                    temp = str(mention["text"])
                    temp = temp.replace(username,"")
                    comments.append(temp)
        
        data=[[tweetId,parent_created_at,parent_user_name,parent_Tweet_Title,parent_tweet_text,parent_tweet_URL,parent_view_count,parent_reply_count,parent_like_count,parent_retweet_count,APP_TYPE]]
        df = pd.DataFrame(data,columns=["ACTIVITY_ID","PUBLISHED_DATE","AUTHOR_NAME","TITLE","TEXT","POST_URL","VIEWS","COMMENTS","LIKES","SHARES","APPLICATION_TYPE"
])
        twitterDF = twitterDF.append(df)

    return twitterDF

## Output

Available Columns / Data :

- Text
- Created At
- Public Metrics (Retweet Count, Reply Count,Like Count)
- List of users who retweeted 
- List of users who liked the tweet
- Comments with mentions

More details can be found at https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/user

In [125]:
get_post_stats(tweetIdList)

,ACTIVITY_ID,PUBLISHED_DATE,AUTHOR_NAME,TITLE,TEXT,POST_URL,VIEWS,COMMENTS,LIKES,SHARES,APPLICATION_TYPE
0,1512912532633235456,09-04-2022 21:57:10,elonmusk,,@teslaownersSV The resemblance is uncanny http...,https://twitter.com/elonmusk/status/1512912532...,0,537,4223,367,Twitter
0,1512900245507776515,09-04-2022 21:08:21,elonmusk,,"@LPofMN As the saying goes, when the governmen...",https://twitter.com/elonmusk/status/1512900245...,0,859,14437,1767,Twitter
0,1512895884719267844,09-04-2022 20:51:01,elonmusk,,@siddarthpaim Fake news purveyors would have h...,https://twitter.com/elonmusk/status/1512895884...,0,411,3108,211,Twitter
0,1512895152288935946,09-04-2022 20:48:07,elonmusk,,@universal_sci All of Earth is a tiny dust mot...,https://twitter.com/elonmusk/status/1512895152...,0,1554,14262,693,Twitter
0,1512892870272991236,09-04-2022 20:39:03,elonmusk,,@UKSAguy @RubinReport Yes,https://twitter.com/elonmusk/status/1512892870...,0,182,943,52,Twitter
0,1512887644128260109,09-04-2022 20:18:17,elonmusk,,@JohnnaCrider1 .@Beeple is 🔥🔥,https://twitter.com/elonmusk/status/1512887644...,0,354,4866,210,Twitter
0,1512886651940491270,09-04-2022 20:14:20,elonmusk,,69.420% of statistics are false,https://twitter.com/elonmusk/status/1512886651...,0,16297,238013,24343,Twitter
0,1512886157876600833,09-04-2022 20:12:22,elonmusk,,Truth is the first casualty. https://t.co/cN9P...,https://twitter.com/elonmusk/status/1512886157...,0,6011,59044,7509,Twitter
0,1512865842194751495,09-04-2022 18:51:39,elonmusk,,@Jason @justinbieber Yeah,https://twitter.com/elonmusk/status/1512865842...,0,254,1466,50,Twitter
0,1512813698011836422,09-04-2022 15:24:26,elonmusk,,RT @Tesla: Thank you to everyone who came out ...,https://twitter.com/elonmusk/status/1512813698...,0,0,0,2679,Twitter
